# NetsPresso reparameterization


### What is Reparameterization ?
Reparameterization is used to reduce trainable BoF modules into deploy model for fast inference. For example merge BN to conv, merge YOLOR to conv, ..etc
However, before reparameterization, the model has more parameters and computation cost.reparameterized model (cfg/deploy) used for deployment purpose



### Steps required for model conversion.
1.train custom model & you will get your own weight i.e custom_weight.pt / use (pretrained weight which is available i.e yolov7_traing.pt)

2.Converting this weight using  Reparameterization  method.

3.Trained model (cfg/training) and reparameterized model (cfg/deploy) will get same prediction results.
However, before reparameterization, the model has more parameters and computation cost.

4.Convert reparameterized weight into onnx & tensorrt
For faster inference & deployment purpose.

## Compressed YOLOv7 reparameterization

In [ ]:
# import
from copy import deepcopy
from models.yolo import Model
import torch
from torch import nn
from utils.torch_utils import select_device, is_parallel
import yaml

device = select_device('0', batch_size=1)
# model trained by cfg/training/*.yaml
ckpt = torch.load('cfg/retraining/yolov7_training.pt', map_location=device)
# reparameterized model in cfg/deploy/*.yaml
model = Model('cfg/deploy/yolov7.yaml', ch=3, nc=80).to(device)

with open('cfg/deploy/yolov7.yaml') as f:
    yml = yaml.load(f, Loader=yaml.SafeLoader)
anchors = len(yml['anchors'][0]) // 2

state_dict = ckpt['model'].float().state_dict()

# check NetsPresso FD
for i in state_dict:
    assert (f'model.105.m' in i and 'netspressofds' in i) == False, 'Reparameterization is not possible after using NetsPresso FD because the model structure has changed'

# copy intersect weights
exclude = []
check_layer = []

for k, v in state_dict.items():
    if k in model.state_dict() and not any(x in k for x in exclude):
        name_k = k.split('.')
        
        if ''.join(name_k[1:-1]) not in  check_layer:
            check_layer.append(''.join(name_k[1:-1]))
            
            if len(name_k) == 4:
                if isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], torch.nn.modules.conv.Conv2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].in_channels = v.size(1)
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].out_channels = v.size(0)
                elif isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], torch.nn.modules.batchnorm.BatchNorm2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].num_features = v.size(0)
                setattr(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], name_k[3], nn.Parameter(v)) 
            
            elif len(name_k) == 5:
                if isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], torch.nn.modules.conv.Conv2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].in_channels = v.size(1)
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].out_channels = v.size(0)
                elif isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], torch.nn.modules.batchnorm.BatchNorm2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].num_features = v.size(0)
                setattr(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], name_k[4], nn.Parameter(v))

model.names = ckpt['model'].names
model.nc = ckpt['model'].nc

# reparametrized YOLOR
for i in range((model.nc+5)*anchors):
    model.state_dict()['model.105.m.0.weight'].data[i, :, :, :] *= state_dict['model.105.im.0.implicit'].data[:, i, : :].squeeze()
    model.state_dict()['model.105.m.1.weight'].data[i, :, :, :] *= state_dict['model.105.im.1.implicit'].data[:, i, : :].squeeze()
    model.state_dict()['model.105.m.2.weight'].data[i, :, :, :] *= state_dict['model.105.im.2.implicit'].data[:, i, : :].squeeze()
model.state_dict()['model.105.m.0.bias'].data += state_dict['model.105.m.0.weight'].mul(state_dict['model.105.ia.0.implicit']).sum(1).squeeze()
model.state_dict()['model.105.m.1.bias'].data += state_dict['model.105.m.1.weight'].mul(state_dict['model.105.ia.1.implicit']).sum(1).squeeze()
model.state_dict()['model.105.m.2.bias'].data += state_dict['model.105.m.2.weight'].mul(state_dict['model.105.ia.2.implicit']).sum(1).squeeze()
model.state_dict()['model.105.m.0.bias'].data *= state_dict['model.105.im.0.implicit'].data.squeeze()
model.state_dict()['model.105.m.1.bias'].data *= state_dict['model.105.im.1.implicit'].data.squeeze()
model.state_dict()['model.105.m.2.bias'].data *= state_dict['model.105.im.2.implicit'].data.squeeze()

# model to be saved
ckpt = {'model': deepcopy(model.module if is_parallel(model) else model).half(),
        'optimizer': None,
        'training_results': None,
        'epoch': -1}

# save reparameterized model
torch.save(ckpt, 'cfg/deploy/yolov7.pt')

## Compressed YOLOv7x reparameterization

In [ ]:
# import
from copy import deepcopy
from models.yolo import Model
import torch
from torch import nn
from utils.torch_utils import select_device, is_parallel
import yaml

device = select_device('0', batch_size=1)
# model trained by cfg/training/*.yaml
ckpt = torch.load('cfg/retraining/yolov7x_trainig.pt', map_location=device)
# reparameterized model in cfg/deploy/*.yaml
model = Model('cfg/deploy/yolov7x.yaml', ch=3, nc=80).to(device)

with open('cfg/deploy/yolov7x.yaml') as f:
    yml = yaml.load(f, Loader=yaml.SafeLoader)
anchors = len(yml['anchors'][0]) // 2

state_dict = ckpt['model'].float().state_dict()

# check NetsPresso FD
for i in state_dict:
    assert (f'model.121.m' in i and 'netspressofds' in i) == False, 'Reparameterization is not possible after using NetsPresso FD because the model structure has changed'
    
# copy intersect weights   
exclude = []
check_layer = []

for k, v in state_dict.items():
    if k in model.state_dict() and not any(x in k for x in exclude):
        name_k = k.split('.')
        
        if ''.join(name_k[1:-1]) not in  check_layer:
            check_layer.append(''.join(name_k[1:-1]))
            
            if len(name_k) == 4:
                if isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], torch.nn.modules.conv.Conv2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].in_channels = v.size(1)
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].out_channels = v.size(0)
                elif isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], torch.nn.modules.batchnorm.BatchNorm2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].num_features = v.size(0)
                setattr(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], name_k[3], nn.Parameter(v)) 
            
            elif len(name_k) == 5:
                if isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], torch.nn.modules.conv.Conv2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].in_channels = v.size(1)
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].out_channels = v.size(0)
                elif isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], torch.nn.modules.batchnorm.BatchNorm2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].num_features = v.size(0)
                setattr(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], name_k[4], nn.Parameter(v))

model.names = ckpt['model'].names
model.nc = ckpt['model'].nc

# reparametrized YOLOR
for i in range((model.nc+5)*anchors):
    model.state_dict()['model.121.m.0.weight'].data[i, :, :, :] *= state_dict['model.121.im.0.implicit'].data[:, i, : :].squeeze()
    model.state_dict()['model.121.m.1.weight'].data[i, :, :, :] *= state_dict['model.121.im.1.implicit'].data[:, i, : :].squeeze()
    model.state_dict()['model.121.m.2.weight'].data[i, :, :, :] *= state_dict['model.121.im.2.implicit'].data[:, i, : :].squeeze()
model.state_dict()['model.121.m.0.bias'].data += state_dict['model.121.m.0.weight'].mul(state_dict['model.121.ia.0.implicit']).sum(1).squeeze()
model.state_dict()['model.121.m.1.bias'].data += state_dict['model.121.m.1.weight'].mul(state_dict['model.121.ia.1.implicit']).sum(1).squeeze()
model.state_dict()['model.121.m.2.bias'].data += state_dict['model.121.m.2.weight'].mul(state_dict['model.121.ia.2.implicit']).sum(1).squeeze()
model.state_dict()['model.121.m.0.bias'].data *= state_dict['model.121.im.0.implicit'].data.squeeze()
model.state_dict()['model.121.m.1.bias'].data *= state_dict['model.121.im.1.implicit'].data.squeeze()
model.state_dict()['model.121.m.2.bias'].data *= state_dict['model.121.im.2.implicit'].data.squeeze()

# model to be saved
ckpt = {'model': deepcopy(model.module if is_parallel(model) else model).half(),
        'optimizer': None,
        'training_results': None,
        'epoch': -1}

# save reparameterized model
torch.save(ckpt, 'cfg/deploy/yolov7x.pt')


## Compressed YOLOv7-W6 reparameterization

In [ ]:
# import
from copy import deepcopy
from models.yolo import Model
import torch
from torch import nn
from utils.torch_utils import select_device, is_parallel
import yaml

device = select_device('0', batch_size=1)
# model trained by cfg/training/*.yaml
ckpt = torch.load('cfg/retraining/yolov7-w6_trainig.pt', map_location=device)
# reparameterized model in cfg/deploy/*.yaml
model = Model('cfg/deploy/yolov7-w6.yaml', ch=3, nc=80).to(device)

with open('cfg/deploy/yolov7-w6.yaml') as f:
    yml = yaml.load(f, Loader=yaml.SafeLoader)
anchors = len(yml['anchors'][0]) // 2

idx = 118
idx2 = 122

# copy intersect weights
state_dict = ckpt['model'].float().state_dict()

# check NetsPresso FD
for i in state_dict:
    assert (f'model.{idx2}.m' in i and 'netspressofds' in i) == False, 'Reparameterization is not possible after using NetsPresso FD because the model structure has changed'
    
# copy intersect weights
exclude = []
check_layer = []

for k, v in state_dict.items():
    if k in model.state_dict() and not any(x in k for x in exclude):
        name_k = k.split('.')
        if name_k[1] == idx:
            break
        
        if ''.join(name_k[1:-1]) not in  check_layer:
            check_layer.append(''.join(name_k[1:-1]))
            
            if len(name_k) == 4:
                if isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], torch.nn.modules.conv.Conv2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].in_channels = v.size(1)
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].out_channels = v.size(0)
                elif isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], torch.nn.modules.batchnorm.BatchNorm2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].num_features = v.size(0)
                setattr(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], name_k[3], nn.Parameter(v)) 
            
            elif len(name_k) == 5:
                if isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], torch.nn.modules.conv.Conv2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].in_channels = v.size(1)
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].out_channels = v.size(0)
                elif isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], torch.nn.modules.batchnorm.BatchNorm2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].num_features = v.size(0)
                setattr(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], name_k[4], nn.Parameter(v))

model.names = ckpt['model'].names
model.nc = ckpt['model'].nc

# copy weights of lead head
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].in_channel = state_dict['model.{}.m.0.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].out_channel = state_dict['model.{}.m.0.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].weight.data = state_dict['model.{}.m.0.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].bias.data = state_dict['model.{}.m.0.bias'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].in_channel = state_dict['model.{}.m.1.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].out_channel = state_dict['model.{}.m.1.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].weight.data = state_dict['model.{}.m.1.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].bias.data = state_dict['model.{}.m.1.bias'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].in_channel = state_dict['model.{}.m.2.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].out_channel = state_dict['model.{}.m.2.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].weight.data = state_dict['model.{}.m.2.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].bias.data = state_dict['model.{}.m.2.bias'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].in_channel = state_dict['model.{}.m.3.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].out_channel = state_dict['model.{}.m.3.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].weight.data = state_dict['model.{}.m.3.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].bias.data = state_dict['model.{}.m.3.bias'.format(idx2)].data

# reparametrized YOLOR
for i in range((model.nc+5)*anchors):
    model.state_dict()['model.{}.m.0.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.0.implicit'.format(idx2)].data[:, i, : :].squeeze()
    model.state_dict()['model.{}.m.1.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.1.implicit'.format(idx2)].data[:, i, : :].squeeze()
    model.state_dict()['model.{}.m.2.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.2.implicit'.format(idx2)].data[:, i, : :].squeeze()
    model.state_dict()['model.{}.m.3.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.3.implicit'.format(idx2)].data[:, i, : :].squeeze()
model.state_dict()['model.{}.m.0.bias'.format(idx)].data += state_dict['model.{}.m.0.weight'.format(idx2)].mul(state_dict['model.{}.ia.0.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.1.bias'.format(idx)].data += state_dict['model.{}.m.1.weight'.format(idx2)].mul(state_dict['model.{}.ia.1.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.2.bias'.format(idx)].data += state_dict['model.{}.m.2.weight'.format(idx2)].mul(state_dict['model.{}.ia.2.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.3.bias'.format(idx)].data += state_dict['model.{}.m.3.weight'.format(idx2)].mul(state_dict['model.{}.ia.3.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.0.bias'.format(idx)].data *= state_dict['model.{}.im.0.implicit'.format(idx2)].data.squeeze()
model.state_dict()['model.{}.m.1.bias'.format(idx)].data *= state_dict['model.{}.im.1.implicit'.format(idx2)].data.squeeze()
model.state_dict()['model.{}.m.2.bias'.format(idx)].data *= state_dict['model.{}.im.2.implicit'.format(idx2)].data.squeeze()
model.state_dict()['model.{}.m.3.bias'.format(idx)].data *= state_dict['model.{}.im.3.implicit'.format(idx2)].data.squeeze()

# model to be saved
ckpt = {'model': deepcopy(model.module if is_parallel(model) else model).half(),
        'optimizer': None,
        'training_results': None,
        'epoch': -1}

# save reparameterized model
torch.save(ckpt, 'cfg/deploy/yolov7-w6.pt')


## Compressed YOLOv7-E6 reparameterization

In [ ]:
# import
from copy import deepcopy
from models.yolo import Model
import torch
from torch import nn
from utils.torch_utils import select_device, is_parallel
import yaml

device = select_device('0', batch_size=1)
# model trained by cfg/training/*.yaml
ckpt = torch.load('cfg/retraining/yolov7-e6.pt', map_location=device)
# reparameterized model in cfg/deploy/*.yaml
model = Model('cfg/deploy/yolov7-e6.yaml', ch=3, nc=80).to(device)

with open('cfg/deploy/yolov7-e6.yaml') as f:
    yml = yaml.load(f, Loader=yaml.SafeLoader)
anchors = len(yml['anchors'][0]) // 2

idx = 140
idx2 = 144

state_dict = ckpt['model'].float().state_dict()

# check NetsPresso FD
for i in state_dict:
    assert (f'model.{idx2}.m' in i and 'netspressofds' in i) == False, 'Reparameterization is not possible after using NetsPresso FD because the model structure has changed'
    
# copy intersect weights
exclude = []
check_layer = []

for k, v in state_dict.items():
    if k in model.state_dict() and not any(x in k for x in exclude):
        name_k = k.split('.')
        if name_k[1] == idx:
            break
        
        if ''.join(name_k[1:-1]) not in  check_layer:
            check_layer.append(''.join(name_k[1:-1]))
            
            if len(name_k) == 4:
                if isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], torch.nn.modules.conv.Conv2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].in_channels = v.size(1)
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].out_channels = v.size(0)
                elif isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], torch.nn.modules.batchnorm.BatchNorm2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].num_features = v.size(0)
                setattr(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], name_k[3], nn.Parameter(v)) 
            
            elif len(name_k) == 5:
                if isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], torch.nn.modules.conv.Conv2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].in_channels = v.size(1)
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].out_channels = v.size(0)
                elif isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], torch.nn.modules.batchnorm.BatchNorm2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].num_features = v.size(0)
                setattr(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], name_k[4], nn.Parameter(v))

model.names = ckpt['model'].names
model.nc = ckpt['model'].nc

# copy weights of lead head
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].in_channel = state_dict['model.{}.m.0.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].out_channel = state_dict['model.{}.m.0.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].weight.data = state_dict['model.{}.m.0.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].bias.data = state_dict['model.{}.m.0.bias'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].in_channel = state_dict['model.{}.m.1.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].out_channel = state_dict['model.{}.m.1.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].weight.data = state_dict['model.{}.m.1.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].bias.data = state_dict['model.{}.m.1.bias'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].in_channel = state_dict['model.{}.m.2.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].out_channel = state_dict['model.{}.m.2.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].weight.data = state_dict['model.{}.m.2.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].bias.data = state_dict['model.{}.m.2.bias'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].in_channel = state_dict['model.{}.m.3.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].out_channel = state_dict['model.{}.m.3.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].weight.data = state_dict['model.{}.m.3.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].bias.data = state_dict['model.{}.m.3.bias'.format(idx2)].data

# reparametrized YOLOR
for i in range((model.nc+5)*anchors):
    model.state_dict()['model.{}.m.0.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.0.implicit'.format(idx2)].data[:, i, : :].squeeze()
    model.state_dict()['model.{}.m.1.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.1.implicit'.format(idx2)].data[:, i, : :].squeeze()
    model.state_dict()['model.{}.m.2.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.2.implicit'.format(idx2)].data[:, i, : :].squeeze()
    model.state_dict()['model.{}.m.3.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.3.implicit'.format(idx2)].data[:, i, : :].squeeze()
model.state_dict()['model.{}.m.0.bias'.format(idx)].data += state_dict['model.{}.m.0.weight'.format(idx2)].mul(state_dict['model.{}.ia.0.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.1.bias'.format(idx)].data += state_dict['model.{}.m.1.weight'.format(idx2)].mul(state_dict['model.{}.ia.1.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.2.bias'.format(idx)].data += state_dict['model.{}.m.2.weight'.format(idx2)].mul(state_dict['model.{}.ia.2.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.3.bias'.format(idx)].data += state_dict['model.{}.m.3.weight'.format(idx2)].mul(state_dict['model.{}.ia.3.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.0.bias'.format(idx)].data *= state_dict['model.{}.im.0.implicit'.format(idx2)].data.squeeze()
model.state_dict()['model.{}.m.1.bias'.format(idx)].data *= state_dict['model.{}.im.1.implicit'.format(idx2)].data.squeeze()
model.state_dict()['model.{}.m.2.bias'.format(idx)].data *= state_dict['model.{}.im.2.implicit'.format(idx2)].data.squeeze()
model.state_dict()['model.{}.m.3.bias'.format(idx)].data *= state_dict['model.{}.im.3.implicit'.format(idx2)].data.squeeze()

# model to be saved
ckpt = {'model': deepcopy(model.module if is_parallel(model) else model).half(),
        'optimizer': None,
        'training_results': None,
        'epoch': -1}

# save reparameterized model
torch.save(ckpt, 'cfg/deploy/yolov7-e6.pt')


## Compressed YOLOv7-D6 reparameterization

In [ ]:
# import
from copy import deepcopy
from models.yolo import Model
import torch
from torch import nn
from utils.torch_utils import select_device, is_parallel
import yaml

device = select_device('0', batch_size=1)
# model trained by cfg/training/*.yaml
ckpt = torch.load('cfg/retraining/yolov7-d6_trainig.pt', map_location=device)
# reparameterized model in cfg/deploy/*.yaml
model = Model('cfg/deploy/yolov7-d6.yaml', ch=3, nc=80).to(device)

with open('cfg/deploy/yolov7-d6.yaml') as f:
    yml = yaml.load(f, Loader=yaml.SafeLoader)
anchors = len(yml['anchors'][0]) // 2

idx = 162
idx2 = 166

state_dict = ckpt['model'].float().state_dict()

# check NetsPresso FD
for i in state_dict:
    assert (f'model.{idx2}.m' in i and 'netspressofds' in i) == False, 'Reparameterization is not possible after using NetsPresso FD because the model structure has changed'
    
# copy intersect weights
exclude = []
check_layer = []

for k, v in state_dict.items():
    if k in model.state_dict() and not any(x in k for x in exclude):
        name_k = k.split('.')
        if name_k[1] == idx:
            break
        
        if ''.join(name_k[1:-1]) not in  check_layer:
            check_layer.append(''.join(name_k[1:-1]))
            
            if len(name_k) == 4:
                if isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], torch.nn.modules.conv.Conv2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].in_channels = v.size(1)
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].out_channels = v.size(0)
                elif isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], torch.nn.modules.batchnorm.BatchNorm2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].num_features = v.size(0)
                setattr(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], name_k[3], nn.Parameter(v)) 
            
            elif len(name_k) == 5:
                if isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], torch.nn.modules.conv.Conv2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].in_channels = v.size(1)
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].out_channels = v.size(0)
                elif isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], torch.nn.modules.batchnorm.BatchNorm2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].num_features = v.size(0)
                setattr(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], name_k[4], nn.Parameter(v))

model.names = ckpt['model'].names
model.nc = ckpt['model'].nc

# copy weights of lead head
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].in_channel = state_dict['model.{}.m.0.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].out_channel = state_dict['model.{}.m.0.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].weight.data = state_dict['model.{}.m.0.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].bias.data = state_dict['model.{}.m.0.bias'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].in_channel = state_dict['model.{}.m.1.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].out_channel = state_dict['model.{}.m.1.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].weight.data = state_dict['model.{}.m.1.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].bias.data = state_dict['model.{}.m.1.bias'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].in_channel = state_dict['model.{}.m.2.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].out_channel = state_dict['model.{}.m.2.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].weight.data = state_dict['model.{}.m.2.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].bias.data = state_dict['model.{}.m.2.bias'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].in_channel = state_dict['model.{}.m.3.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].out_channel = state_dict['model.{}.m.3.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].weight.data = state_dict['model.{}.m.3.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].bias.data = state_dict['model.{}.m.3.bias'.format(idx2)].data

# reparametrized YOLOR
for i in range((model.nc+5)*anchors):
    model.state_dict()['model.{}.m.0.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.0.implicit'.format(idx2)].data[:, i, : :].squeeze()
    model.state_dict()['model.{}.m.1.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.1.implicit'.format(idx2)].data[:, i, : :].squeeze()
    model.state_dict()['model.{}.m.2.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.2.implicit'.format(idx2)].data[:, i, : :].squeeze()
    model.state_dict()['model.{}.m.3.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.3.implicit'.format(idx2)].data[:, i, : :].squeeze()
model.state_dict()['model.{}.m.0.bias'.format(idx)].data += state_dict['model.{}.m.0.weight'.format(idx2)].mul(state_dict['model.{}.ia.0.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.1.bias'.format(idx)].data += state_dict['model.{}.m.1.weight'.format(idx2)].mul(state_dict['model.{}.ia.1.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.2.bias'.format(idx)].data += state_dict['model.{}.m.2.weight'.format(idx2)].mul(state_dict['model.{}.ia.2.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.3.bias'.format(idx)].data += state_dict['model.{}.m.3.weight'.format(idx2)].mul(state_dict['model.{}.ia.3.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.0.bias'.format(idx)].data *= state_dict['model.{}.im.0.implicit'.format(idx2)].data.squeeze()
model.state_dict()['model.{}.m.1.bias'.format(idx)].data *= state_dict['model.{}.im.1.implicit'.format(idx2)].data.squeeze()
model.state_dict()['model.{}.m.2.bias'.format(idx)].data *= state_dict['model.{}.im.2.implicit'.format(idx2)].data.squeeze()
model.state_dict()['model.{}.m.3.bias'.format(idx)].data *= state_dict['model.{}.im.3.implicit'.format(idx2)].data.squeeze()

# model to be saved
ckpt = {'model': deepcopy(model.module if is_parallel(model) else model).half(),
        'optimizer': None,
        'training_results': None,
        'epoch': -1}

# save reparameterized model
torch.save(ckpt, 'cfg/deploy/yolov7-d6.pt')


## Compressed YOLOv7-E6E reparameterization

In [ ]:
# import
from copy import deepcopy
from models.yolo import Model
import torch
from torch import nn
from utils.torch_utils import select_device, is_parallel
import yaml

device = select_device('0', batch_size=1)
# model trained by cfg/training/*.yaml
ckpt = torch.load('cfg/retraining/yolov7-e6e_trainig.pt', map_location=device)
# reparameterized model in cfg/deploy/*.yaml
model = Model('cfg/deploy/yolov7-e6e.yaml', ch=3, nc=80).to(device)

with open('cfg/deploy/yolov7-e6e.yaml') as f:
    yml = yaml.load(f, Loader=yaml.SafeLoader)
anchors = len(yml['anchors'][0]) // 2

idx = 261
idx2 = 265

state_dict = ckpt['model'].float().state_dict()

# check NetsPresso FD
for i in state_dict:
    assert (f'model.{idx2}.m' in i and 'netspressofds' in i) == False, 'Reparameterization is not possible after using NetsPresso FD because the model structure has changed'
    
# copy intersect weights
exclude = []
check_layer = []

for k, v in state_dict.items():
    if k in model.state_dict() and not any(x in k for x in exclude):
        name_k = k.split('.')
        if name_k[1] == idx:
            break
        
        if ''.join(name_k[1:-1]) not in  check_layer:
            check_layer.append(''.join(name_k[1:-1]))
            
            if len(name_k) == 4:
                if isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], torch.nn.modules.conv.Conv2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].in_channels = v.size(1)
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].out_channels = v.size(0)
                elif isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], torch.nn.modules.batchnorm.BatchNorm2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]].num_features = v.size(0)
                setattr(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]], name_k[3], nn.Parameter(v)) 
            
            elif len(name_k) == 5:
                if isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], torch.nn.modules.conv.Conv2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].in_channels = v.size(1)
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].out_channels = v.size(0)
                elif isinstance(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], torch.nn.modules.batchnorm.BatchNorm2d):
                    model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]].num_features = v.size(0)
                setattr(model._modules[name_k[0]]._modules[name_k[1]]._modules[name_k[2]]._modules[name_k[3]], name_k[4], nn.Parameter(v))

model.names = ckpt['model'].names
model.nc = ckpt['model'].nc

# copy weights of lead head
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].in_channel = state_dict['model.{}.m.0.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].out_channel = state_dict['model.{}.m.0.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].weight.data = state_dict['model.{}.m.0.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['0'].bias.data = state_dict['model.{}.m.0.bias'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].in_channel = state_dict['model.{}.m.1.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].out_channel = state_dict['model.{}.m.1.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].weight.data = state_dict['model.{}.m.1.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['1'].bias.data = state_dict['model.{}.m.1.bias'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].in_channel = state_dict['model.{}.m.2.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].out_channel = state_dict['model.{}.m.2.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].weight.data = state_dict['model.{}.m.2.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['2'].bias.data = state_dict['model.{}.m.2.bias'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].in_channel = state_dict['model.{}.m.3.weight'.format(idx2)].size(1)
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].out_channel = state_dict['model.{}.m.3.weight'.format(idx2)].size(0)
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].weight.data = state_dict['model.{}.m.3.weight'.format(idx2)].data
model._modules['model']._modules[str(idx)]._modules['m']._modules['3'].bias.data = state_dict['model.{}.m.3.bias'.format(idx2)].data

# reparametrized YOLOR
for i in range((model.nc+5)*anchors):
    model.state_dict()['model.{}.m.0.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.0.implicit'.format(idx2)].data[:, i, : :].squeeze()
    model.state_dict()['model.{}.m.1.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.1.implicit'.format(idx2)].data[:, i, : :].squeeze()
    model.state_dict()['model.{}.m.2.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.2.implicit'.format(idx2)].data[:, i, : :].squeeze()
    model.state_dict()['model.{}.m.3.weight'.format(idx)].data[i, :, :, :] *= state_dict['model.{}.im.3.implicit'.format(idx2)].data[:, i, : :].squeeze()
model.state_dict()['model.{}.m.0.bias'.format(idx)].data += state_dict['model.{}.m.0.weight'.format(idx2)].mul(state_dict['model.{}.ia.0.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.1.bias'.format(idx)].data += state_dict['model.{}.m.1.weight'.format(idx2)].mul(state_dict['model.{}.ia.1.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.2.bias'.format(idx)].data += state_dict['model.{}.m.2.weight'.format(idx2)].mul(state_dict['model.{}.ia.2.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.3.bias'.format(idx)].data += state_dict['model.{}.m.3.weight'.format(idx2)].mul(state_dict['model.{}.ia.3.implicit'.format(idx2)]).sum(1).squeeze()
model.state_dict()['model.{}.m.0.bias'.format(idx)].data *= state_dict['model.{}.im.0.implicit'.format(idx2)].data.squeeze()
model.state_dict()['model.{}.m.1.bias'.format(idx)].data *= state_dict['model.{}.im.1.implicit'.format(idx2)].data.squeeze()
model.state_dict()['model.{}.m.2.bias'.format(idx)].data *= state_dict['model.{}.im.2.implicit'.format(idx2)].data.squeeze()
model.state_dict()['model.{}.m.3.bias'.format(idx)].data *= state_dict['model.{}.im.3.implicit'.format(idx2)].data.squeeze()

# model to be saved
ckpt = {'model': deepcopy(model.module if is_parallel(model) else model).half(),
        'optimizer': None,
        'training_results': None,
        'epoch': -1}

# save reparameterized model
torch.save(ckpt, 'cfg/deploy/yolov7-e6e.pt')
